<a href="https://colab.research.google.com/github/baskayj/Boundary-behavior-in-stochastic-differential-equations-used-in-Finance/blob/master/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
import json
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp

from collections import namedtuple
from google.colab import drive

mc_rep = 10000                                                                  #Number of generated trajectories

In [0]:
#To load and save simulation data

drive.mount('/content/gdrive')
DATA_PATH = '/content/gdrive/My Drive/MSc. Szakdolgozat/Data/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#CIR MODEL
#There are multiple finite-diffrence schemes for simulation including,
#Euler-Maryama Scheme: X[i] = X[i-1] + kappa*(theta-X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt)
#Weighted Milstein Scheme: X[i] = (X[i-1] + kappa*(theta-alpha*X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt) + (1/4)*np.power(sigma,2)*dt*(np.power(W[i-1],2)-1))/(1+(1-alpha)*kappa*dt)
#To be continued...

#gamma = 2*kappa*theta/sigma^2

#Path(returns the entire path as a NumPy array)
def CIR_path(X0, dt, T, kappa, theta, sigma, seed):
  N = int(T/dt)
  alpha = 0.8
  np.random.seed(seed)
  W = np.random.normal(size=N)
  X = np.zeros(N)
  X[0] = X0
  for i in range(1,N):
    x = (X[i-1] + kappa*(theta-alpha*X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt) + (1/4)*np.power(sigma,2)*dt*(np.power(W[i-1],2)-1))/(1+(1-alpha)*kappa*dt)
    if x < 0 :
      X[i] = 0
    else:
      X[i] = x
  return X

#Process(returns only the values at t = T)
def CIR_prcss(X0, dt, T, kappa, theta, sigma, seed):
  N = int(T/dt)
  alpha = 0.8
  np.random.seed(seed)
  W = np.random.normal(size=N)
  X = np.zeros(N)
  X[0] = X0
  for i in range(1,N):
    x = (X[i-1] + kappa*(theta-alpha*X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt) + (1/4)*np.power(sigma,2)*dt*(np.power(W[i-1],2)-1))/(1+(1-alpha)*kappa*dt)
    if x < 0 :
      X[i] = 0
    else:
      X[i] = x
  return X[N-1]

#Eta function, for the Non-central chi^2 method
def eta(dt,kappa,sigma):
  return ((4*kappa)/(np.power(sigma,2)))*np.exp(-kappa*dt)*(1-np.exp(-kappa*dt))

#Non-central chi^2 method
def CIR_prcss_nccs(X0, dt, T, kappa, theta, sigma, seed):
  k = ((4*kappa*theta)/(np.power(sigma,2)))
  N = int(T/dt)
  np.random.seed(seed)
  X = np.zeros(N)
  X[0] = X0
  for i in range(1,N):
    x = np.random.noncentral_chisquare(k,X[i-1]*eta(dt,kappa,sigma)) * np.exp(-kappa*dt) *(1/eta(dt,kappa,sigma))
    if x < 0 :
      X[i] = 0
    else:
      X[i] = x
  return X[N-1]

In [0]:
# CLASSES for MP

#CIR
class CIR():
  def __init__(self,X0,dt,T,kappa,theta,sigma):
    self.X0 = X0
    self.dt = dt
    self.T = T
    self.kappa = kappa
    self.theta = theta
    self.sigma = sigma
  def Path(self,seed):
    return CIR_path(self.X0, self.dt, self.T, self.kappa, self.theta, self.sigma, seed)
  def Prcss(self,seed):
    return CIR_prcss(self.X0, self.dt, self.T, self.kappa, self.theta, self.sigma, seed)
  def Prcss_nccs(self,seed):
    return CIR_prcss_nccs(self.X0, self.dt, self.T, self.kappa, self.theta, self.sigma, seed)

In [0]:
#MULTIPROCESS FUNCTIONS

def CIR_Path_mp(X0,dt,T,kappa,theta,sigma):
  cir = CIR(X0,dt,T,kappa,theta,sigma)
  pool = mp.Pool(processes=mp.cpu_count())
  return pool.map(cir.Path,range(mc_rep))

def CIR_Prcss_mp(X0,dt,T,kappa,theta,sigma):
  cir = CIR(X0,dt,T,kappa,theta,sigma)
  pool = mp.Pool(processes=mp.cpu_count())
  return pool.map(cir.Prcss,range(mc_rep))

def CIR_Prcss_nccs_mp(X0,dt,T,kappa,theta,sigma):
  cir = CIR(X0,dt,T,kappa,theta,sigma)
  pool = mp.Pool(processes=mp.cpu_count())
  return pool.map(cir.Prcss_nccs,range(mc_rep))

In [0]:
#SAVING

def print_log(string):
  logs = open(f"{DATA_PATH}logs.txt","a")
  logs.write(string)
  logs.close()

def Save(model,name,var_name,P,var,X):

  with open(f"{DATA_PATH}{model}{name}_params.txt","w") as outfile1:
    json.dump(P._asdict(),outfile1)
  outfile1.close()

  for i in range(len(var_name)):
    with open(f"{DATA_PATH}{model}{name}_{var_name[i]}_values.txt","w") as outfile2:
      json.dump(var[i],outfile2)
    outfile2.close()
  
  np.save(f"{DATA_PATH}{model}{name}.npy",X)

  print_log(f"Data saved to \"{DATA_PATH}\" succefully.\n")

------

# Relationship between the CIR-Process and the Noncentral $\chi ^2$-distribution

A noncentral $\chi^2$-distribution can be described with the probability density function:
$$f_\chi (x,k,\lambda) = \sum_{x = 0}^{\infty}{\frac{e^{-\lambda/2}(\lambda/2)^i}{i!}f_{Y_k+2i}(x)}$$
Where $Y_q$ is $\chi^2$ distributed with degrees of freedom of $q$.
The parameters of this thistribution:
* $k$ degrees of freedom
* $\lambda$ noncentrality parameter

According to [2] the CIR process can be simulated by setting the degrees of freedom and non-centrality parameters as follows:
* $k := \frac{4\kappa\theta}{\sigma^2} = 2\gamma$
* $\lambda := X_{t_n}\eta(dt)$
Where we introduced the function $\eta(dt)$ as 
$$ \eta(dt) := \frac{4\kappa}{\sigma^2} e^{-\kappa dt} (1-e^{-\kappa dt})$$

With this the $t_{n+1}$-th step looks like:
$$ X_{t_{n+1}} = \chi^2_k (\lambda) \frac{e^{-\kappa dt}}{\eta(dt)} $$

We can use **numpy.random.noncentral_chisquare** to draw random numbers from the noncentral $\chi^2$-distribution.


References:

[1] https://en.wikipedia.org/wiki/Noncentral_chi-squared_distribution

[2] https://arxiv.org/pdf/0802.4411.pdf

---

# EXPERIMENT: $T \rightarrow \infty$ and $X_0 \rightarrow 0$

In [0]:
#EXPERIMENT:T->infty
#T = [1,10,70,700]
#dt = [0.1,0.01,0.001,0.0001]
#X0 = [0.1,0.01,0.001,0.0001]
#gamma = [0.9,1.1]

model = 'CIR_'

name = 'T_to_infty_part_1'

print_log(f"\n\nStaring Experiment {model}{name}.\n")

#Parameters:
Parameters = namedtuple('Parameters','X0 dt T kappa theta sigma')
P = Parameters(X0 = 0.5, dt = (1/10000), T = 1,kappa = 1, theta = 1, sigma = 0.6)

#Variables:
var_name1 = 'T'
var1 = [1,7]

var_name2 = 'dt'
var2 = [0.1,0.01,0.001]

var_name3 = 'X0'
var3 = [0.1,0.01,0.001,0.0001]

var_name4 = 'gamma'
var4 = [0.9,1.1]
sigma = list(map(lambda p: np.sqrt((2*P.kappa*P.theta)/p),var4))

#Transforming the parameters into a more digestable form. This way the computation heavy part of the code needs less for cycles.
T = []
for i in range(len(var1)):
  for j in range(len(var2)):
    for k in range(len(var3)):
      for l in range(len(var4)):
        T.append([var3[k],var2[j],var1[i],P.kappa,P.theta,sigma[l]])

start_time = time.time()
#GENERATING ALL TRAJECTORIES
X = np.zeros((len(T),mc_rep))
for i in range(len(T)):
  print_log(f"Making trajectories for X0 = {T[i][0]},dt = {T[i][1]},T = {T[i][2]},kappa = {T[i][3]},theta = {T[i][4]},sigma = {T[i][5]}.\n")
  s_time = time.time()
  n = 0
  for x in CIR_Prcss_mp(T[i][0],T[i][1],T[i][2],T[i][3],T[i][4],T[i][5]):
    X[i,n] = x
    n += 1
  e_time = time.time()
  print_logprint_log(f"  Finished generating trajectories in {int((e_time-s_time)/60)} minutes and {e_time-s_time-int((e_time-s_time)/60)*60} seconds.[{i+1}/{len(T)}]\n")

#Saving the Data
var_name = [var_name1,var_name2,var_name3,var_name4]
var = [var1,var2,var3,var4]
Save(model,name,var_name,P,var,X)

#Memory Management
del T
del X

end_time = time.time()
print_log(f"The experiment finished in {int((end_time-start_time)/60)} minutes and {end_time-start_time-int((end_time-start_time)/60)*60} seconds.\n")


Staring simulations for X0 = 0.1,dt = 0.1,T = 1,kappa = 1,theta = 1,sigma = 1.4907119849998598.

  Finished generating trajectories in 0 minutes and 0.5932111740112305 seconds.


Staring simulations for X0 = 0.1,dt = 0.1,T = 1,kappa = 1,theta = 1,sigma = 1.348399724926484.

  Finished generating trajectories in 0 minutes and 0.5983624458312988 seconds.


Staring simulations for X0 = 0.01,dt = 0.1,T = 1,kappa = 1,theta = 1,sigma = 1.4907119849998598.

  Finished generating trajectories in 0 minutes and 0.5842022895812988 seconds.


Staring simulations for X0 = 0.01,dt = 0.1,T = 1,kappa = 1,theta = 1,sigma = 1.348399724926484.

  Finished generating trajectories in 0 minutes and 0.6040263175964355 seconds.


Staring simulations for X0 = 0.001,dt = 0.1,T = 1,kappa = 1,theta = 1,sigma = 1.4907119849998598.

  Finished generating trajectories in 0 minutes and 0.5930178165435791 seconds.


Staring simulations for X0 = 0.001,dt = 0.1,T = 1,kappa = 1,theta = 1,sigma = 1.348399724926484.

  Fi

In [0]:
#EXPERIMENT:T->infty
#T = [1,10,70,700]
#dt = [0.1,0.01,0.001,0.0001]
#X0 = [0.1,0.01,0.001,0.0001]
#gamma = [0.9,1.1]

model = 'CIR_'

name = 'T_to_infty_part_2'

#Parameters:
Parameters = namedtuple('Parameters','X0 dt T kappa theta sigma')
P = Parameters(X0 = 0.5, dt = (1/10000), T = 1,kappa = 1, theta = 1, sigma = 0.6)

#Variables:
var_name1 = 'T'
var1 = [1,7]

var_name2 = 'dt'
var2 = [0.0001]

var_name3 = 'X0'
var3 = [0.1,0.01,0.001,0.0001]

var_name4 = 'gamma'
var4 = [0.9,1.1]
sigma = list(map(lambda p: np.sqrt((2*P.kappa*P.theta)/p),var4))

#Transforming the parameters into a more digestable form. This way the computation heavy part of the code needs less for cycles.
T = []
for i in range(len(var1)):
  for j in range(len(var2)):
    for k in range(len(var3)):
      for l in range(len(var4)):
        T.append([var3[k],var2[j],var1[i],P.kappa,P.theta,sigma[l]])

start_time = time.time()
#GENERATING ALL TRAJECTORIES
X = np.zeros((len(T),mc_rep))
for i in range(len(T)):
  print_log(f"Making trajectories for X0 = {T[i][0]},dt = {T[i][1]},T = {T[i][2]},kappa = {T[i][3]},theta = {T[i][4]},sigma = {T[i][5]}.\n")
  s_time = time.time()
  n = 0
  for x in CIR_Prcss_mp(T[i][0],T[i][1],T[i][2],T[i][3],T[i][4],T[i][5]):
    X[i,n] = x
    n += 1
  e_time = time.time()
  print_logprint_log(f"  Finished generating trajectories in {int((e_time-s_time)/60)} minutes and {e_time-s_time-int((e_time-s_time)/60)*60} seconds.[{i+1}/{len(T)}]\n")

#Saving the Data
var_name = [var_name1,var_name2,var_name3,var_name4]
var = [var1,var2,var3,var4]
Save(model,name,var_name,P,var,X)

#Memory Management
del T
del X

end_time = time.time()
print_log(f"The experiment finished in {int((end_time-start_time)/60)} minutes and {end_time-start_time-int((end_time-start_time)/60)*60} seconds.\n")


Staring simulations for X0 = 0.1,dt = 0.0001,T = 1,kappa = 1,theta = 1,sigma = 1.4907119849998598.

  Finished generating trajectories in 8 minutes and 37.630459785461426 seconds.[0/16]


Staring simulations for X0 = 0.1,dt = 0.0001,T = 1,kappa = 1,theta = 1,sigma = 1.348399724926484.

  Finished generating trajectories in 9 minutes and 2.5705535411834717 seconds.[1/16]


Staring simulations for X0 = 0.01,dt = 0.0001,T = 1,kappa = 1,theta = 1,sigma = 1.4907119849998598.

  Finished generating trajectories in 9 minutes and 0.8453657627105713 seconds.[2/16]


Staring simulations for X0 = 0.01,dt = 0.0001,T = 1,kappa = 1,theta = 1,sigma = 1.348399724926484.

  Finished generating trajectories in 9 minutes and 0.2885870933532715 seconds.[3/16]


Staring simulations for X0 = 0.001,dt = 0.0001,T = 1,kappa = 1,theta = 1,sigma = 1.4907119849998598.

  Finished generating trajectories in 8 minutes and 54.956379413604736 seconds.[4/16]


Staring simulations for X0 = 0.001,dt = 0.0001,T = 1,kapp

In [0]:
#EXPERIMENT:T->infty
#T = [1,10,70,700]
#dt = [0.1,0.01,0.001,0.0001]
#X0 = [0.1,0.01,0.001,0.0001]
#gamma = [0.9,1.1]

model = 'CIR_'

name = 'T_to_infty_part_3'

#Parameters:
Parameters = namedtuple('Parameters','X0 dt T kappa theta sigma')
P = Parameters(X0 = 0.5, dt = (1/10000), T = 1,kappa = 1, theta = 1, sigma = 0.6)

#Variables:
var_name1 = 'T'
var1 = [70]

var_name2 = 'dt'
var2 = [0.1,0.01,0.001]

var_name3 = 'X0'
var3 = [0.1,0.01,0.001,0.0001]

var_name4 = 'gamma'
var4 = [0.9,1.1]
sigma = list(map(lambda p: np.sqrt((2*P.kappa*P.theta)/p),var4))

#Transforming the parameters into a more digestable form. This way the computation heavy part of the code needs less for cycles.
T = []
for i in range(len(var1)):
  for j in range(len(var2)):
    for k in range(len(var3)):
      for l in range(len(var4)):
        T.append([var3[k],var2[j],var1[i],P.kappa,P.theta,sigma[l]])

start_time = time.time()
#GENERATING ALL TRAJECTORIES
X = np.zeros((len(T),mc_rep))
for i in range(len(T)):
  print_log(f"Making trajectories for X0 = {T[i][0]},dt = {T[i][1]},T = {T[i][2]},kappa = {T[i][3]},theta = {T[i][4]},sigma = {T[i][5]}.\n")
  s_time = time.time()
  n = 0
  for x in CIR_Prcss_mp(T[i][0],T[i][1],T[i][2],T[i][3],T[i][4],T[i][5]):
    X[i,n] = x
    n += 1
  e_time = time.time()
  print_logprint_log(f"  Finished generating trajectories in {int((e_time-s_time)/60)} minutes and {e_time-s_time-int((e_time-s_time)/60)*60} seconds.[{i+1}/{len(T)}]\n")

#Saving the Data
var_name = [var_name1,var_name2,var_name3,var_name4]
var = [var1,var2,var3,var4]
Save(model,name,var_name,P,var,X)

#Memory Management
del T
del X

end_time = time.time()
print_log(f"The experiment finished in {int((end_time-start_time)/60)} minutes and {end_time-start_time-int((end_time-start_time)/60)*60} seconds.\n")


Staring simulations for X0 = 0.1,dt = 0.1,T = 70,kappa = 1,theta = 1,sigma = 1.4907119849998598.

  Finished generating trajectories in 0 minutes and 37.115392446517944 seconds.[(0+1)/24]


Staring simulations for X0 = 0.1,dt = 0.1,T = 70,kappa = 1,theta = 1,sigma = 1.348399724926484.

  Finished generating trajectories in 0 minutes and 39.281275510787964 seconds.[(1+1)/24]


Staring simulations for X0 = 0.01,dt = 0.1,T = 70,kappa = 1,theta = 1,sigma = 1.4907119849998598.

  Finished generating trajectories in 0 minutes and 38.542348861694336 seconds.[(2+1)/24]


Staring simulations for X0 = 0.01,dt = 0.1,T = 70,kappa = 1,theta = 1,sigma = 1.348399724926484.

  Finished generating trajectories in 0 minutes and 38.49348473548889 seconds.[(3+1)/24]


Staring simulations for X0 = 0.001,dt = 0.1,T = 70,kappa = 1,theta = 1,sigma = 1.4907119849998598.

  Finished generating trajectories in 0 minutes and 39.01931071281433 seconds.[(4+1)/24]


Staring simulations for X0 = 0.001,dt = 0.1,T = 7

In [0]:
#EXPERIMENT:T->infty
#T = [1,10,70,700]
#dt = [0.1,0.01,0.001,0.0001]
#X0 = [0.1,0.01,0.001,0.0001]
#gamma = [0.9,1.1]

model = 'CIR_'

name = 'T_to_infty_part_4'

#Parameters:
Parameters = namedtuple('Parameters','X0 dt T kappa theta sigma')
P = Parameters(X0 = 0.5, dt = (1/10000), T = 1,kappa = 1, theta = 1, sigma = 0.6)

#Variables:
var_name1 = 'T'
var1 = [700]

var_name2 = 'dt'
var2 = [0.1,0.01]

var_name3 = 'X0'
var3 = [0.1,0.01,0.001,0.0001]

var_name4 = 'gamma'
var4 = [0.9,1.1]
sigma = list(map(lambda p: np.sqrt((2*P.kappa*P.theta)/p),var4))

#Transforming the parameters into a more digestable form. This way the computation heavy part of the code needs less for cycles.
T = []
for i in range(len(var1)):
  for j in range(len(var2)):
    for k in range(len(var3)):
      for l in range(len(var4)):
        T.append([var3[k],var2[j],var1[i],P.kappa,P.theta,sigma[l]])

start_time = time.time()
#GENERATING ALL TRAJECTORIES
X = np.zeros((len(T),mc_rep))
for i in range(len(T)):
  print_log(f"Making trajectories for X0 = {T[i][0]},dt = {T[i][1]},T = {T[i][2]},kappa = {T[i][3]},theta = {T[i][4]},sigma = {T[i][5]}.\n")
  s_time = time.time()
  n = 0
  for x in CIR_Prcss_mp(T[i][0],T[i][1],T[i][2],T[i][3],T[i][4],T[i][5]):
    X[i,n] = x
    n += 1
  e_time = time.time()
  print_logprint_log(f"  Finished generating trajectories in {int((e_time-s_time)/60)} minutes and {e_time-s_time-int((e_time-s_time)/60)*60} seconds.[{i+1}/{len(T)}]\n")

#Saving the Data
var_name = [var_name1,var_name2,var_name3,var_name4]
var = [var1,var2,var3,var4]
Save(model,name,var_name,P,var,X)

#Memory Management
del T
del X

end_time = time.time()
print_log(f"The experiment finished in {int((end_time-start_time)/60)} minutes and {end_time-start_time-int((end_time-start_time)/60)*60} seconds.\n")


Staring simulations for X0 = 0.1,dt = 0.1,T = 700,kappa = 1,theta = 1,sigma = 1.4907119849998598.

  Finished generating trajectories in 6 minutes and 25.889183282852173 seconds.[1/16]


Staring simulations for X0 = 0.1,dt = 0.1,T = 700,kappa = 1,theta = 1,sigma = 1.348399724926484.

  Finished generating trajectories in 6 minutes and 42.84909701347351 seconds.[2/16]


Staring simulations for X0 = 0.01,dt = 0.1,T = 700,kappa = 1,theta = 1,sigma = 1.4907119849998598.

  Finished generating trajectories in 6 minutes and 41.64670014381409 seconds.[3/16]


Staring simulations for X0 = 0.01,dt = 0.1,T = 700,kappa = 1,theta = 1,sigma = 1.348399724926484.

  Finished generating trajectories in 6 minutes and 40.78744149208069 seconds.[4/16]


Staring simulations for X0 = 0.001,dt = 0.1,T = 700,kappa = 1,theta = 1,sigma = 1.4907119849998598.

  Finished generating trajectories in 6 minutes and 42.56146454811096 seconds.[5/16]


Staring simulations for X0 = 0.001,dt = 0.1,T = 700,kappa = 1,thet

# EXPERIMENT: Using the Noncentral $\chi ^2$-distribution

In [0]:
#EXPERIMENT:Simulating the process using the noncentral chi^2-distribution

model = 'CIR_'

name = 'nccs_test'

print_log(f"\n\nStaring Experiment {model}{name}.\n")

#Parameters:
Parameters = namedtuple('Parameters','X0 dt T kappa theta sigma')
P = Parameters(X0 = 0.5, dt = (1/100), T = 1,kappa = 1, theta = 1, sigma = 0.6)

#Variables:
var_name = 'gamma'
var = [-1,0.01,0.9,1.1,2]

#kappa = list(map(lambda p: p*np.power(P.sigma,2)*(1/(2*P.theta)),var))
#theta = list(map(lambda p: p*np.power(P.sigma,2)*(1/(2*P.kappa)),var))
sigma = list(map(lambda p: np.sqrt((2*P.kappa*P.theta)/p),var))

#Transforming the parameters into a more digestable form. This way the computation heavy part of the code needs less for cycles.
T = []
for i in range(len(var)):
  T.append([P.X0,P.dt,P.T,P.kappa,P.theta,sigma[i]])

start_time = time.time()
#GENERATING ALL TRAJECTORIES
X = np.zeros((len(T),mc_rep))
for i in range(len(T)):
  print_log(f"Making trajectories for X0 = {T[i][0]},dt = {T[i][1]},T = {T[i][2]},kappa = {T[i][3]},theta = {T[i][4]},sigma = {T[i][5]}.\n")

  s_time = time.time()
  n = 0
  for x in CIR_Prcss_nccs_mp(T[i][0],T[i][1],T[i][2],T[i][3],T[i][4],T[i][5]):
    X[i,n] = x
    n += 1
  e_time = time.time()

  print_log(f"  Finished generating trajectories in {int((e_time-s_time)/60)} minutes and {e_time-s_time-int((e_time-s_time)/60)*60} seconds.[{i+1}/{len(T)}]\n")

#Saving the Data
var_name = [var_name]
var = [var]
Save(model,name,var_name,P,var,X)

#Memory Management
del T
del X

end_time = time.time()

print_log(f"The experiment finished in {int((end_time-start_time)/60)} minutes and {end_time-start_time-int((end_time-start_time)/60)*60} seconds.\n")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in sqrt


#EXPERIMENT: Finding the critical $\gamma$-value numerically

# MISC. TEST

In [0]:
#EXPERIMENT:

model = 'CIR_'

name = 'test'

print_log(f"\n\nStaring Experiment {model}{name}.\n")

#Parameters:
Parameters = namedtuple('Parameters','X0 dt T kappa theta sigma')
P = Parameters(X0 = 0.5, dt = (1/100), T = 1,kappa = 1, theta = 1, sigma = 0.6)

#Variables:
var_name = 'gamma'
var = [-1,0.01,0.9,1.1,2]

#kappa = list(map(lambda p: p*np.power(P.sigma,2)*(1/(2*P.theta)),var))
#theta = list(map(lambda p: p*np.power(P.sigma,2)*(1/(2*P.kappa)),var))
sigma = list(map(lambda p: np.sqrt((2*P.kappa*P.theta)/p),var))

#Transforming the parameters into a more digestable form. This way the computation heavy part of the code needs less for cycles.
T = []
for i in range(len(var)):
  T.append([P.X0,P.dt,P.T,P.kappa,P.theta,sigma[i]])

start_time = time.time()
#GENERATING ALL TRAJECTORIES
X = np.zeros((len(T),mc_rep))
for i in range(len(T)):
  print_log(f"Making trajectories for X0 = {T[i][0]},dt = {T[i][1]},T = {T[i][2]},kappa = {T[i][3]},theta = {T[i][4]},sigma = {T[i][5]}.\n")

  s_time = time.time()
  n = 0
  for x in CIR_Prcss_mp(T[i][0],T[i][1],T[i][2],T[i][3],T[i][4],T[i][5]):
    X[i,n] = x
    n += 1
  e_time = time.time()

  print_log(f"  Finished generating trajectories in {int((e_time-s_time)/60)} minutes and {e_time-s_time-int((e_time-s_time)/60)*60} seconds.[{i+1}/{len(T)}]\n")

#Saving the Data
var_name = [var_name]
var = [var]
Save(model,name,var_name,P,var,X)

#Memory Management
del T
del X

end_time = time.time()

print_log(f"The experiment finished in {int((end_time-start_time)/60)} minutes and {end_time-start_time-int((end_time-start_time)/60)*60} seconds.\n")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in sqrt
